<a href="https://colab.research.google.com/github/amzad-786githumb/AI_and_ML_by-Microsoft/blob/main/2_Auditing_ML_code_for_security_vulnerabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the required libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle
import hashlib

In [ ]:
#Validate and sanatize input data

def validate_data(df):
  if df.isnull().values.any():
    raise ValueError("Dataset contains null values. Please clean the data before processing")
  return df

In [ ]:
#load the validated dataset
df =  validate_data(pd.read_csv("/content/customer_churn.csv"))

In [ ]:
df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,Cameron Williams,42.0,11066.80,0,7.22,8.0,2013-08-30 07:00:40,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1
1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,2013-08-13 00:38:46,"6157 Frank Gardens Suite 019 Carloshaven, RI 1...",Wilson PLC,1
2,Eric Lozano,38.0,12884.75,0,6.67,12.0,2016-06-29 06:20:07,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1
3,Phillip White,42.0,8010.76,0,6.71,10.0,2014-04-22 12:43:12,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1
4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,2016-01-19 15:31:15,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1


In [ ]:
from sklearn.preprocessing import StandardScaler
from scipy import stats

def load_data(df):
    return df

def handle_missing_values(df):
    # Select only numeric columns before calculating the mean
    numeric_cols = df.select_dtypes(include=np.number).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
    return df

def remove_outliers(df):
    numeric_cols = df.select_dtypes(include=[np.number])
    z_scores = np.abs(stats.zscore(numeric_cols))
    # Create a boolean mask for rows to keep
    mask = (z_scores < 3).all(axis=1)
    # Apply the mask to the original dataframe
    return df[mask]

def scale_data(df):
    scaler = StandardScaler()
    # Exclude the 'Churn' column from scaling
    numeric_cols = df.select_dtypes(include=[np.number]).columns.drop('Churn', errors='ignore')
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    return df

def encode_categorical(df, categorical_columns):
    return pd.get_dummies(df, columns=categorical_columns)

def save_data(df, output_filepath):
    df.to_csv(output_filepath, index=False)

In [ ]:
# Load the data
df_preprocessed = load_data(df)

# Display columns to help identify categorical variables
print("Columns in the dataframe:")
print(df_preprocessed.columns)

# Handle missing values
df_preprocessed = handle_missing_values(df_preprocessed)

# Remove outliers
df_preprocessed = remove_outliers(df_preprocessed)

# Scale the data
df_preprocessed = scale_data(df_preprocessed)

# Encode categorical variables (update 'Category' to actual categorical columns if needed)
# df_preprocessed = encode_categorical(df_preprocessed, ['Category']) # Removed as 'Category' column does not exist

# Display the preprocessed data
print("\nPreprocessed data head:")
print(df_preprocessed.head())

Columns in the dataframe:
Index(['Names', 'Age', 'Total_Purchase', 'Account_Manager', 'Years',
       'Num_Sites', 'Onboard_date', 'Location', 'Company', 'Churn'],
      dtype='object')

Preprocessed data head:
              Names       Age  ...                      Company  Churn
0  Cameron Williams  0.030149  ...                   Harvey LLC      1
1     Kevin Mueller -0.134738  ...                   Wilson PLC      1
2       Eric Lozano -0.629398  ...  Miller, Johnson and Wallace      1
3     Phillip White  0.030149  ...                    Smith Inc      1
4    Cynthia Norton -0.794285  ...                   Love-Jones      1

[5 rows x 10 columns]


/tmp/ipython-input-3877790145.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [ ]:
#split the dataset

X = df_preprocessed.iloc[:,:-1]
y = df_preprocessed.iloc[:,-1]

# Identify categorical columns
categorical_columns = ['Names', 'Location', 'Company', 'Onboard_date']

# Encode categorical variables
X = encode_categorical(X, categorical_columns)

# Display columns after encoding
print("Columns in X after encoding:")
print(X.columns)

Columns in X after encoding:
Index(['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites',
       'Names_Aaron King', 'Names_Aaron Meyer', 'Names_Aaron West',
       'Names_Abigail Gonzalez', 'Names_Abigail Jennings',
       ...
       'Onboard_date_2016-11-07 08:00:39', 'Onboard_date_2016-11-07 08:01:15',
       'Onboard_date_2016-11-13 13:13:01', 'Onboard_date_2016-11-17 09:21:44',
       'Onboard_date_2016-11-17 14:37:01', 'Onboard_date_2016-11-21 11:16:13',
       'Onboard_date_2016-12-05 03:35:43', 'Onboard_date_2016-12-07 02:57:32',
       'Onboard_date_2016-12-09 09:29:39', 'Onboard_date_2016-12-28 04:07:38'],
      dtype='object', length=3521)


In [ ]:
X

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Names_Aaron King,Names_Aaron Meyer,Names_Aaron West,Names_Abigail Gonzalez,Names_Abigail Jennings,Names_Adam Gomez,Names_Adam Harris,Names_Adam Moore,Names_Adam Porter,Names_Adam Strong,Names_Adam Waters,Names_Adriana James,Names_Alan Gray,Names_Albert Clayton,Names_Alex Banks,Names_Alexis Allen,Names_Alexis Hill,Names_Allison Clark DDS,Names_Alyssa Harper DDS,Names_Alyssa Pitts,Names_Amanda Barnett,Names_Amanda Brown,Names_Amanda Hernandez,Names_Amanda Moore,Names_Amanda Roberts,Names_Amanda Wade,Names_Amanda Williams,Names_Amber Carr,Names_Amber Evans,Names_Amber Hobbs,Names_Amber Martin,Names_Amber Spears,Names_Amber Williams,Names_Amy Baldwin,Names_Amy Garcia,...,Onboard_date_2016-04-22 21:07:11,Onboard_date_2016-05-09 00:40:38,Onboard_date_2016-05-12 17:50:46,Onboard_date_2016-05-14 14:37:08,Onboard_date_2016-05-24 04:02:24,Onboard_date_2016-05-26 20:27:02,Onboard_date_2016-05-29 06:00:09,Onboard_date_2016-06-08 11:44:18,Onboard_date_2016-06-19 01:23:45,Onboard_date_2016-06-22 19:20:04,Onboard_date_2016-06-24 08:32:37,Onboard_date_2016-06-29 06:20:07,Onboard_date_2016-07-04 20:08:46,Onboard_date_2016-07-21 02:39:27,Onboard_date_2016-07-29 08:39:49,Onboard_date_2016-07-29 12:56:50,Onboard_date_2016-08-06 00:18:54,Onboard_date_2016-08-10 20:11:33,Onboard_date_2016-08-14 18:04:40,Onboard_date_2016-08-22 05:47:48,Onboard_date_2016-09-09 03:56:57,Onboard_date_2016-09-18 16:37:16,Onboard_date_2016-09-27 07:00:35,Onboard_date_2016-09-30 14:28:53,Onboard_date_2016-10-02 23:17:33,Onboard_date_2016-10-05 16:14:32,Onboard_date_2016-10-06 23:11:34,Onboard_date_2016-10-12 21:52:56,Onboard_date_2016-10-12 23:20:12,Onboard_date_2016-11-06 01:13:12,Onboard_date_2016-11-07 08:00:39,Onboard_date_2016-11-07 08:01:15,Onboard_date_2016-11-13 13:13:01,Onboard_date_2016-11-17 09:21:44,Onboard_date_2016-11-17 14:37:01,Onboard_date_2016-11-21 11:16:13,Onboard_date_2016-12-05 03:35:43,Onboard_date_2016-12-07 02:57:32,Onboard_date_2016-12-09 09:29:39,Onboard_date_2016-12-28 04:07:38
0,0.030149,0.423272,-0.962334,1.542362,-0.336689,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,-0.134738,0.780104,-0.962334,0.972390,1.435430,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,-0.629398,1.186974,-0.962334,1.106966,2.026136,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,0.030149,-0.860540,-0.962334,1.138632,0.844724,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [ ]:
#split the data into training and testing set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Train a logistic regression model

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# Save the model to disk with encryption
filename = 'finalized_model.sav'
with open(filename, 'wb') as model_file:
    encrypted_model = pickle.dumps(model)
    model_file.write(encrypted_model)

In [ ]:
# Load the model from disk and verify its integrity
with open(filename, 'rb') as model_file:
    loaded_model = pickle.loads(model_file.read())
    if hashlib.sha256(pickle.dumps(loaded_model)).hexdigest() != hashlib.sha256(pickle.dumps(model)).hexdigest():
        raise ValueError("Model integrity check failed. The model may have been tampered with.")
result = loaded_model.score(X_test, y_test)
print(f'Model Accuracy: {result:.2f}')

Model Accuracy: 0.87
